# Part I. ETL Pipeline for Pre-Processing the Files

### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # Print the list of Files under the Root directory
    for file in file_path_list:
        print(file)

D:\Data Engineer\Udacity\Data Modeling with Cassandra
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-01-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-02-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-03-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-04-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-05-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-06-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-07-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-08-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-09-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-10-events.csv
D:\Data Engineer\Udacity\Data Modeling with Cassandra/event_data\2018-11-11-

### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        # Iterate to the first row (Column Header) to skip the header
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            # Add the line to the list
            full_data_rows_list.append(line) 
            
#  get total number of rows 
print("Length of the list containing all the lines of the file(s): ", len(full_data_rows_list))

# creating a event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables containing all the individual csv files
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        # Ignore the records with No Artist Names
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
f.close()

Length of the list containing all the lines of the file(s):  4586


In [16]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("No of Lines in the new CSV file: ", len(f.readlines()))


No of Lines in the new CSV file:  3902


# Part II. Complete the Apache Cassandra coding portion of the project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the current directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

### Creating a Cluster

In [17]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

### Create a Keyspace

In [18]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1 } """
    ) 
except Exception as e:
    print(e)

### Set the Keyspace to the existing session

In [19]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we can model the database tables on the queries we want to run.

## Create queries to ask the following three questions to the data

<ol>
    <b>
<li> Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4 </li>


<li> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 </li>
    

<li> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
</li>
</b>
</ol>

### FIRST QUERY: Create table `song_library` with following fields from `Music App History`

<b>
<ol>
<li> artist </li>
<li> song title </li>
<li> song's length </li>
<li> sessionId </li>
<li> itemInSession</li>
</ol>
</b>

### `SessionId and ItemInSession` are partition keys that can be uniquely identify each row.

![Table for Query1](images/Query_table1.png)

In [26]:
# Create Table
query1 = "CREATE TABLE IF NOT EXISTS song_library"
query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY(sessionId, itemInSession)) "

try:
    session.execute(query1)
except Exception as e:
    print(e)

### Data File containing the consolidated data from the event data log, used to feed all the tables to be created for the above stated business queries.

In [27]:
file = 'event_datafile_new.csv'

### Insert the data to the above `song_library` table

In [28]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (sessionId, itemInsession, artist, song, length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int (line[3]), line[0], line[9], float (line[5])))   
        

#### VALIDATE THE DATA MODEL BY USING THE SAMPLE QUERY PROVIDED BY THE BUSINESS

In [32]:
# Query the data was entered into the table
query = "SELECT artist, song, length FROM song_library where sessionId=338 and itemInSession=4"
try:
    rows= session.execute(query)
    
    print(disp_df(rows))
    
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)
    

      artist                             song    length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073


### SECOND QUERY: Create table `artist_library` with following fields from `Music App History`

<b>
<ol>
<li> artist </li>
<li> song title </li>
<li> user first name </li>
<li> user last name </li>
<li> userid </li>
<li> session id </li>
<li> itemInSession </li>
</ol>
</b>

#### `UserID and SessionId` are partition keys and the `itemInSession` is the clustering Key , together they form the Primary Key that can be used to uniquely identify each row.

![Table for Query2](images/Query_table2.png)

In [36]:
# Create Table 
query2 = "CREATE TABLE IF NOT EXISTS artist_library"
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId),itemInSession)) "

try:
    session.execute(query2)
except Exception as e:
    print(e)


### Insert the data to the above `artist_library` table

In [37]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int (line[8]), int(line[3]), line[0], line[9],  line[1], line[4])) 

#### VALIDATE THE DATA MODEL BY USING THE SAMPLE QUERY PROVIDED BY THE BUSINESS

In [40]:
# Query the artist data for selection userId=10 and sessionId=182
query = "SELECT artist, song, firstName, lastName FROM artist_library where userId=10 and sessionId=182"
try:
    rows = session.execute(query)  
    print(disp_df(rows))
    
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)


              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


### THIRD QUERY: Create table `users_library` with following fields from `Music App History`

<b>
<ol>
<li> user first name </li>
<li> user last name </li>
<li> song title </li>
<li> userid </li>
</ol>
</b>

#### `UserID and Song` are partition keys that can be used to uniquely identify each row.

![Table for Query3](images/Query_table3.png)

In [43]:
# Query to create the users library table
query3 = "CREATE TABLE IF NOT EXISTS users_library"
query3 = query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId)) "

try:
    session.execute(query3)
except Exception as e:
    print(e)
                    

### Insert the data to the above `users_library` table

In [44]:
# Insert the data from each CSV file into the above created table users_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_library (song, userId, firstName, lastName)"
        query = query + "VALUES(%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4])) 

#### VALIDATE THE DATA MODEL BY USING THE SAMPLE QUERY PROVIDED BY THE BUSINESS

In [45]:
# Query the data for users who listened to the song 'All Hands Against His Own'
query = "SELECT firstName, lastName FROM users_library where song= 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    print(disp_df(rows))
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [48]:
# Drop the tables before closing out the sessions
dropTable("song_library")
dropTable("artist_library")
dropTable("users_library")

### DISPLAY RESULTSET AS DATAFRAME

In [49]:
def disp_df(rows):
    try:
        
        # Extract the Columns of the ResultSet
        cols = rows.one()._fields
        # COnvert rows into list of lists
        data = [list(row) for row in rows]
        # Convert ResultSet into DataFrame
        df = pd.DataFrame(data,columns=cols)
    
        return df
    except Exception as e:
        return e

### DROP TABLE FUNCTION

In [50]:
# Drop table from the KeySpace
def dropTable(table):
    try:
        query = "DROP TABLE IF EXISTS " + table
        session.execute(query)
        
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [51]:
session.shutdown()
cluster.shutdown()